In [3]:
from scrape_league_data import create_underdog_df_dict
from pull_bearer_token import pull_bearer_token
import getpass
import pandas as pd

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

### Variables to change ###
chromedriver_path = '/usr/bin/chromedriver'
username = input('Enter Underdog username: ')
password = getpass.getpass()

### Keep as is ###
url = "https://underdogfantasy.com/lobby"    
bearer_token = pull_bearer_token(url, chromedriver_path, username, password)

### Pull all major UD data elements ###
# underdog_data = create_underdog_df_dict(bearer_token, sleep_time=5)

In [8]:
underdog_data['df_drafts'].to_csv('df_drafts.csv', index=False)
underdog_data['df_league_info'].to_csv('df_league_info.csv', index=False)

In [7]:
import time
import os
import json
from requests import JSONDecodeError

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import UD_draft_model.scrapers.scrape_site.scrape_league_data as scrape_site


def create_webdriver(url, chromedriver_path, username, password):

    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

    driver = webdriver.Chrome(chromedriver_path)
    driver.get(url)

    # elem = driver.find_elements_by_class_name('styles__field__3fmc7')[0]
    elem = driver.find_elements_by_class_name("styles__field__OeiFa  ")[0]
    elem.clear()
    elem.send_keys(username)
    elem.send_keys(Keys.RETURN)

    # elem = driver.find_elements_by_class_name('styles__field__3fmc7')[0]
    elem = driver.find_elements_by_class_name("styles__field__OeiFa  ")[1]
    elem.clear()
    elem.send_keys(password)
    elem.send_keys(Keys.RETURN)

    return driver


def pull_logs(url, chromedriver_path, username, password):

    driver = create_webdriver(url, chromedriver_path, username, password)

    time.sleep(5)
    logs = driver.get_log("performance")

    driver.close()
    driver.quit()

    return logs


def pull_bearer_token(logs):

    for log in logs:
        try:
            log_dict = json.loads(log["message"])
            bearer_token = log_dict["message"]["params"]["response"]["headers"][
                "authorization"
            ]

            if bearer_token[:6] == "Bearer":
                break

        except:
            pass

    return bearer_token


def pull_user_agent(logs):

    for log in logs:
        try:
            log_dict = json.loads(log["message"])
            val = log_dict["message"]["params"]["request"]["headers"][
                "User-Agent"
            ]
            break
        except:
            pass

    return val


def pull_required_headers(url, chromedriver_path, username, password):

    logs = pull_logs(url, chromedriver_path, username, password)

    bearer_token = pull_bearer_token(logs)
    user_agent = pull_user_agent(logs)

    headers = {
        'authorization': bearer_token,
        'user-agent': user_agent
    }

    return headers


def create_headers_path() -> str:
    """
    Creates the path to the bearer_token data.
    """

    relative_path = "scrapers/scrape_site/bearer_token"

    cwd = os.getcwd()
    folder_end_index = cwd.rfind("UD_draft_model") + len("UD_draft_model")

    if cwd.rfind("UD_draft_model") == -1:
        base_path = os.path.join(cwd, "UD_draft_model")
    else:
        base_path = cwd[:folder_end_index]

    token_dir = os.path.join(base_path, relative_path)
    token_path = os.path.join(token_dir, "bearer_token.json")

    return token_path


def read_headers(file_path: str = None) -> dict:
    """
    Reads in all user's bearer tokens.
    """

    if file_path is None:
        file_path = create_headers_path()

    with open(file_path, "r") as f:
        json_data = json.load(f)

    return json_data


def save_headers(username: str, headers: dict) -> None:
    """
    Saves the bearer token to a json file to prevent re-scraping the token
    when it's still active.
    """

    token_path = create_headers_path()

    try:
        header_data = read_headers(file_path=token_path)
    except FileNotFoundError:
        header_data = {}

    header_data[username] = headers

    with open(token_path, "w") as f:
        json.dump(header_data, f)

    return None


def test_headers(headers: dict) -> bool:
    """
    Returns a True value if the bearer_token.
    """

    url = "https://api.underdogfantasy.com/v1/user"
    base_data = scrape_site.BaseData(headers)
    # base_data.auth_header["authorization"] = bearer_token

    # JSONDecodeError thrown when the user-agent is incorrect.
    try:
        data = base_data.read_in_site_data(url, headers=base_data.auth_header)
    except JSONDecodeError:
        token_valid = False

        return token_valid

    # Returns an error in the reponse if the bearer token has expired.
    if "error" in (list(data.keys())):
        token_valid = False
    else:
        token_valid = True

    return token_valid

In [9]:
headers = read_headers()['condelong11@yahoo.com']
headers['authorization'] = 'aaa'
check = test_headers(headers)
check

False

In [3]:
import getpass

### Variables to change ###
chromedriver_path = "/usr/bin/chromedriver"
username = input("Enter Underdog username: ")
password = getpass.getpass()

### Keep as is ###
url = "https://underdogfantasy.com/lobby"
user_agent = pull_user_agent(url, chromedriver_path, username, password)

In [45]:
headers = pull_required_headers(url, chromedriver_path, username, password)

In [47]:
save_headers(username, headers)